### Simple tutorial for parsing from the `BISTRO` Dashboard database<br/>
Few things to note: <br/>
1. Adopted from Robert's master branch, which mainly serves for parsing to the Dashboard for a few particular submission output visualizations.
2. Currently only have the uploaded Sioux Faux submissions, plannig to upload outputs from sf_light as well.
3. For a complete table structure, please reference the BISTRO ACM paper, while there are inconsistency with how actually the parsing outputs here, it gives a clear structure for manipulating the results.
4. The tables (for example, pathtraversal) is being implemented with the many-to-many relation in our database, which uses pairs of foreign keys to link two or more tables whereas parsing with one-to-one is very time consuming considering we are parsing many submissions on the Dashboard at the same time, which is another reason that not all columns of the table find in the ACM paper is parsed.
5. You might also need to modify or expand a few parsing functions, mainly in [`db_loader.py`](https://github.com/bistro-its-berkeley/BISTRO_Dashboard/blob/e6735e32558ddf04b2b3d6c81df24b936dc465bc/BISTRO_Dashboard/db_loader.py#L17), and reference the schema file [here](https://github.com/bistro-its-berkeley/BISTRO-Starter-Kit/blob/ry-update-visualization/utilities/bistro_dbschema.py)
6. Below are just a few simple examples of the existing parsing functions which you can run after your clone it from my [branch](https://github.com/bistro-its-berkeley/BISTRO_Dashboard/tree/dashboard-jarvis), and feel free to push up any changes that you made.
7. As you might know, there are a few type of travel modes allowed during the Beam simulation (transit, drive, walk-transit, etc.), so you would want to filter your visual up by look into different tables. For example, `trip` contains `legs`, which is made up by a few `paths`, which is then further broken down into `links`.
8. Let me know if there's any questions!

In [1]:
from db_loader import BistroDB
import numpy as np
import pandas as pd

In [2]:
# you should see 'Connected to DB at 13.56.123.155'

database = BistroDB('bistro', 'bistroclt', 'client', '13.56.123.155')

Connected to DB at 13.56.123.155


### sioux faux
- all simulations

In [3]:
submission = '5e2eb80c54eafa4e9a396fc0'
# submission = 'siouxfaux_15k_BAU'
## Currently BAU doesnt have the corresponding links to join with..

# tag = if you want to use a specific tag
simulation = database.load_simulation_df()[database.load_simulation_df()['name'] == submission].reset_index()

simulation_id = simulation.loc[0, 'simulation_id']
scenario = simulation.loc[0, 'scenario']

print('simulation id = ' + simulation_id)
print('scenario = ' + scenario)

# it needs to be put in a list
simulation_id = [simulation_id]
simulation

simulation id = 790458ce-adb2-11ea-9534-063f0fd82f9f
scenario = sioux_faux-15k


,index,simulation_id,datetime,scenario,name,tag
0,1505,790458ce-adb2-11ea-9534-063f0fd82f9f,2020-01-27 10:46:22,sioux_faux-15k,5e2eb80c54eafa4e9a396fc0,None


In [4]:
trips = database.load_trips(simulation_id)
legs = database.load_legs(simulation_id)
# load_legs() are currently grouping the links
legs_w_links = database.load_legs(simulation_id, links=True)

paths = database.load_paths(simulation_id, scenario)
# load_paths() are not currently grouped, you can comment out the two lines from the script, as an example.
paths_w_links = database.load_paths(simulation_id, scenario, links=True)
links = database.load_links(scenario)

person = database.load_person(scenario)
activities = database.load_activities(scenario)

In [15]:
legs.head()

,PID,Trip_ID,Leg_ID,Distance_m,Mode,Veh,Start_time,End_time,Fare,fuelCost,Toll
0,100-2012000131467-02126,1,1,27.440,walk,body-100-2012000131467-02126,29828,29850,0.0,0.0,0.0
1,100-2012000131467-02126,1,2,6.796,walk,body-100-2012000131467-02126,29976,29982,0.0,0.0,0.0
2,100-2012000131467-02126,1,3,3148.275,walk,body-100-2012000131467-02126,29982,32412,0.0,0.0,0.0
3,100-2012000131467-02126,2,1,3354.988,walk,body-100-2012000131467-02126,63152,65742,0.0,0.0,0.0
4,100-2012000131467-02169,1,1,27.440,walk,body-100-2012000131467-02169,29828,29850,0.0,0.0,0.0


In [18]:
legs_w_links.tail()

,PID,Trip_ID,Leg_ID,Distance_m,Mode,Veh,Start_time,End_time,Fare,fuelCost,Toll,LinkId
57411,900-2016001458235-0156103,2,4,256.889,walk,body-900-2016001458235-0156103,73800,74000,0.0,0.000000,0.00,"[575.0, 877.0, 4494.0, 5435.0, 5669.0, 5767.0]"
57412,900-2016001458235-0156129,1,1,2235.055,car,901-2,30944,31594,0.0,0.245139,8.50,"[326.0, 574.0, 879.0, 1037.0, 1149.0, 2146.0, ..."
57413,900-2016001458235-0156129,2,1,1970.715,car,901-2,64458,65824,0.0,0.225388,7.34,"[326.0, 575.0, 827.0, 1148.0, 1465.0, 2383.0, ..."
57414,900-2016001508783-0157630,1,1,1987.527,car,563-1,30170,30636,0.0,0.169282,8.39,"[1100.0, 1464.0, 2072.0, 2106.0, 2891.0, 3310...."
57415,900-2016001508783-0157630,2,1,2239.443,car,563-1,63180,64725,0.0,0.262548,7.58,"[1099.0, 1101.0, 1465.0, 2073.0, 2106.0, 3311...."


In [19]:
links.head()

,LinkId,fromLocationID,toLocationID,fromLocationX,fromLocationY,toLocationX,toLocationY,length,freespeed
0,0,0,1,43.511307,-96.729576,43.511072,-96.731145,129.161,6.17284
1,2481,0,369,43.511307,-96.729576,43.511948,-96.726660,245.555,6.17284
2,1,1,0,43.511072,-96.731145,43.511307,-96.729576,129.161,6.17284
3,634,1,303,43.511072,-96.731145,43.511055,-96.731291,11.876,6.17284
4,742,1,628,43.511072,-96.731145,43.511641,-96.731223,63.600,6.17284


In [20]:
paths_w_links.head()

,vehicle,distance,mode,departureTime,arrivalTime,numPassengers,fuelCost,fuelConsumed,LinkId,vehicleType
0,10-1,1957.646,car,60783,61285,0,0.180594,6.019806e+06,1236,CAR-TYPE-DEFAULT
1,10-1,1957.646,car,60783,61285,0,0.180594,6.019806e+06,1580,CAR-TYPE-DEFAULT
2,10-1,1957.646,car,60783,61285,0,0.180594,6.019806e+06,2101,CAR-TYPE-DEFAULT
3,10-1,1957.646,car,60783,61285,0,0.180594,6.019806e+06,2502,CAR-TYPE-DEFAULT
4,10-1,1957.646,car,60783,61285,0,0.180594,6.019806e+06,2608,CAR-TYPE-DEFAULT


In [21]:
print(trips.shape)
print(len(trips['PID'].unique().tolist()))
trips.head()

(30002, 12)
15001


,PID,realizedTripMode,Distance_m,Trip_ID,Start_time,End_time,fuelCost,Fare,Toll,Incentive,DestinationAct,Duration_sec
0,100-2012000131467-02126,walk,3182.511,1,29828,32412,0.000000,0.0,0.00,0.0,2,2584
1,100-2012000131467-02126,walk,3354.988,2,63152,65742,0.000000,0.0,0.00,0.0,3,2590
2,100-2012000131467-02169,walk,5255.359,1,29828,34016,0.000000,0.0,0.00,0.0,2,4188
3,100-2012000131467-02169,walk_transit,9382.339,2,65635,70168,0.000000,1.5,0.00,0.0,3,4533
4,100-2012000131467-02212,car,2245.114,1,30888,31440,0.198955,0.0,8.09,0.0,2,552


In [5]:
activities.head()

,PID,ActNum,Type
0,100-2012000131467-02126,1,Home
1,100-2012000131467-02126,2,Work
2,100-2012000131467-02126,3,Home
3,100-2012000131467-02169,1,Home
4,100-2012000131467-02169,2,Work


----

## sf_light
- currently only one BAU `sf_light-50k` scenario exists for sf_light for the 06/25 BEAM output
    - simulation_id = `db21069e-d19b-11ea-bfff-faffc250aee5`
    - fixed data was pull from the BISTRO master branch [here](https://github.com/bistro-its-berkeley/BISTRO/tree/master/fixed-data)
    - missing `submission_input data`, defaulted and ignored for now

In [10]:
simulation = database.load_simulation_df()[database.load_simulation_df()['scenario'] == 'sf_light-50k']
simulation

,simulation_id,datetime,scenario,name,tag
1506,db21069e-d19b-11ea-bfff-faffc250aee5,2020-06-25 15:48:27,sf_light-50k,sf_light_BAU,None


In [17]:
simulation_id = 'db21069e-d19b-11ea-bfff-faffc250aee5'# simulation_id for BAU 06/25 BEAM output
scenario = simulation.iloc[0, 2]

print('simulation id = ' + simulation_id)
print('scenario = ' + scenario)
simulation_id = ['db21069e-d19b-11ea-bfff-faffc250aee5']

simulation id = db21069e-d19b-11ea-bfff-faffc250aee5
scenario = sf_light-50k


In [20]:
# takes around 3 minutes depends on the machine and internet connection

trips = database.load_trips(simulation_id)
legs = database.load_legs(simulation_id)
# load_legs() are currently grouping the links
legs_w_links = database.load_legs(simulation_id, links=True)

paths = database.load_paths(simulation_id, scenario)
# load_paths() are not currently grouped, you can comment out the two lines from the script, as an example.
paths_w_links = database.load_paths(simulation_id, scenario, links=True)
links = database.load_links(scenario)

person = database.load_person(scenario)